In [1]:
import json
import time
import uuid
import base64
import hashlib
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime
from enum import Enum
import asyncio


# OpenRTB Enums

In [2]:


class AuctionType(Enum):
    FIRST_PRICE = 1
    SECOND_PRICE = 2

class CreativeStatus(Enum):
    APPROVED = 1
    PENDING_REVIEW = 2
    REJECTED = 3

# Data Classes for OpenRTB
@dataclass
class Banner:
    w: int
    h: int
    format: List[Dict] = None
    
    def __post_init__(self):
        if self.format is None:
            self.format = [{"w": self.w, "h": self.h}]

@dataclass
class Imp:
    id: str
    banner: Banner
    tagid: str
    bidfloor: float = 0.0
    bidfloorcur: str = "USD"
    secure: int = 1
    ext: Dict = None

@dataclass
class Site:
    id: str
    domain: str
    page: str
    name: str = "Publisher"
    cat: List[str] = None
    content: Dict = None

@dataclass
class Device:
    ua: str
    ip: str
    devicetype: int = 1  # 1=mobile, 2=personal computer
    make: str = "Apple"
    model: str = "iPhone"
    os: str = "iOS"

@dataclass
class User:
    id: str
    buyeruid: str = None
    keywords: str = ""

@dataclass
class BidRequest:
    id: str
    imp: List[Imp]
    site: Site
    device: Device
    user: User
    at: int = AuctionType.SECOND_PRICE.value
    tmax: int = 100
    cur: List[str] = None
    test: int = 1
    
    def __post_init__(self):
        if self.cur is None:
            self.cur = ["USD"]

@dataclass
class Bid:
    id: str
    impid: str
    price: float
    adid: str
    adm: str
    ext: Dict = None

@dataclass
class SeatBid:
    seat: str
    bid: List[Bid]

@dataclass
class BidResponse:
    id: str
    seatbid: List[SeatBid]
    bidid: str = None
    
    def __post_init__(self):
        if self.bidid is None:
            self.bidid = str(uuid.uuid4())[:8]


# RTBBidder

In [3]:

class RTBBidder:
    """Real-Time Bidder implementing OpenRTB protocol"""
    
    def __init__(self, bidder_id: str = "bidder-123", buyer_id: str = "buyer-123"):
        self.bidder_id = bidder_id
        self.buyer_id = buyer_id
        self.stats = {
            "bids_submitted": 0,
            "bids_won": 0,
            "total_spend": 0.0,
            "impressions": []
        }
        self.creatives = {
            "banner-1": {
                "id": "banner-1",
                "width": 300,
                "height": 250,
                "markup": "<div style='background:#f0f0f0;padding:20px'>Sample Ad</div>",
                "status": CreativeStatus.APPROVED
            }
        }
    
    def generate_bid_request(self, 
                            publisher_url: str,
                            ad_unit: str,
                            width: int = 300,
                            height: int = 250,
                            bid_floor: float = 0.5) -> BidRequest:
        """Generate OpenRTB BidRequest matching Google's protocol"""
        
        request_id = self._generate_request_id()
        
        banner = Banner(w=width, h=height)
        imp = Imp(
            id="1",
            banner=banner,
            tagid=ad_unit,
            bidfloor=bid_floor,
            ext={"billing_id": [12345]}
        )
        
        site = Site(
            id="site-pub-123",
            domain=publisher_url.split("//")[1].split("/")[0],
            page=publisher_url,
            name="Publisher"
        )
        
        device = Device(
            ua="Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X)",
            ip="192.168.1.100",
            devicetype=1
        )
        
        user = User(
            id=f"user-{uuid.uuid4().hex[:8]}",
            buyeruid=str(uuid.uuid4())
        )
        
        bid_request = BidRequest(
            id=request_id,
            imp=[imp],
            site=site,
            device=device,
            user=user,
            at=AuctionType.SECOND_PRICE.value,
            tmax=100
        )
        
        return bid_request
    
    def evaluate_request(self, bid_request: BidRequest) -> Tuple[float, str]:
        """Apply bidding logic to determine price and creative"""
        
        impression = bid_request.imp[0]
        base_bid = 5.0  # Base CPM in dollars
        
        # Contextual scoring: adjust based on site quality, viewability, etc.
        site_quality_score = self._score_site(bid_request.site)
        viewability_adjustment = 1.0 + (site_quality_score * 0.3)
        
        # Apply viewability metric if available
        if hasattr(impression, 'metric') and impression.metric:
            viewability = impression.metric[0].get('value', 0.5)
            if viewability > 0.5:
                viewability_adjustment *= 1.15
        
        final_bid = base_bid * viewability_adjustment
        final_bid = max(final_bid, impression.bidfloor)
        
        # Select creative
        creative_id = "banner-1"
        
        return round(final_bid, 2), creative_id
    
    def submit_bid(self, 
                   bid_request: BidRequest,
                   bid_price: float,
                   creative_id: str) -> BidResponse:
        """Construct and submit OpenRTB BidResponse"""
        
        impression = bid_request.imp[0]
        creative = self.creatives.get(creative_id)
        
        if not creative:
            raise ValueError(f"Creative {creative_id} not found")
        
        bid = Bid(
            id=str(uuid.uuid4())[:8],
            impid=impression.id,
            price=bid_price,
            adid=creative_id,
            adm=creative["markup"],
            ext={
                "billing_id": 12345,
                "impression_tracking_url": [
                    f"https://track.example.com/imp?auction_id=${{AUCTION_ID}}&price=${{AUCTION_PRICE}}&creative={creative_id}"
                ]
            }
        )
        
        seatbid = SeatBid(seat=self.buyer_id, bid=[bid])
        bid_response = BidResponse(id=bid_request.id, seatbid=[seatbid])
        
        self.stats["bids_submitted"] += 1
        
        return bid_response
    
    def process_bid_feedback(self, 
                            bid_response: BidResponse,
                            feedback: Dict) -> None:
        """Process real-time feedback from Google"""
        
        creative_status = feedback.get("creative_status", 0)
        price = feedback.get("price", 0)
        won = feedback.get("won", False)
        
        if won:
            self.stats["bids_won"] += 1
            self.stats["total_spend"] += price
            self.stats["impressions"].append({
                "timestamp": datetime.now().isoformat(),
                "price": price,
                "creative_status": creative_status,
                "request_id": bid_response.id
            })
    
    def get_performance_metrics(self) -> Dict:
        """Calculate key performance indicators"""
        
        bids = self.stats["bids_submitted"]
        wins = self.stats["bids_won"]
        spend = self.stats["total_spend"]
        
        return {
            "bids_submitted": bids,
            "bids_won": wins,
            "win_rate": (wins / bids * 100) if bids > 0 else 0,
            "total_spend": round(spend, 2),
            "average_cpm": round(spend / wins, 2) if wins > 0 else 0,
            "impressions_count": len(self.stats["impressions"])
        }
    
    # Helper methods
    @staticmethod
    def _generate_request_id() -> str:
        """Generate websafe base64 request ID (no padding)"""
        raw_id = base64.b64encode(uuid.uuid4().bytes)
        websafe = raw_id.decode('ascii').replace('+', '-').replace('/', '_').rstrip('=')
        return websafe
    
    @staticmethod
    def _score_site(site: Site) -> float:
        """Simple site quality scoring (0-1)"""
        score = 0.6
        if "premium" in site.name.lower():
            score += 0.2
        if "https" in site.page:
            score += 0.1
        return min(score, 1.0)
    
    def to_json(self, obj) -> str:
        """Convert OpenRTB objects to JSON"""
        def converter(o):
            if hasattr(o, '__dataclass_fields__'):
                return asdict(o)
            elif isinstance(o, Enum):
                return o.value
            raise TypeError(f"Object of type {o.__class__.__name__} is not JSON serializable")
        
        return json.dumps(obj, default=converter, indent=2)


# Initialize bidder

In [4]:

bidder = RTBBidder(bidder_id="my-bidder", buyer_id="my-buyer")

print("=" * 60)
print("Google Authorized Buyers RTB Bidder - OpenRTB Protocol Demo")
print("=" * 60)


Google Authorized Buyers RTB Bidder - OpenRTB Protocol Demo


In [5]:

# Simulate multiple bid requests
for i in range(50):
    print(f"\n--- Auction {i + 1} ---\n")
    
    # Generate bid request
    bid_request = bidder.generate_bid_request(
        publisher_url="https://example-publisher.com/article",
        ad_unit="sidebar-ad-123",
        width=300,
        height=250,
        bid_floor=1.50
    )
    
    print("BID REQUEST (OpenRTB):")
    print(bidder.to_json(bid_request))
    
    # Evaluate and determine bid
    bid_price, creative_id = bidder.evaluate_request(bid_request)
    print(f"\n→ Decision: Bid ${bid_price} CPM with creative {creative_id}")
    
    # Submit bid response
    bid_response = bidder.submit_bid(bid_request, bid_price, creative_id)
    print("\nBID RESPONSE (OpenRTB):")
    print(bidder.to_json(bid_response))
    
    # Simulate feedback from exchange
    won = True if i % 2 == 0 else False
    feedback = {
        "creative_status": CreativeStatus.APPROVED.value,
        "price": bid_price if won else bid_price + 0.50,
        "won": won
    }
    bidder.process_bid_feedback(bid_response, feedback)
    
    result = "✓ WON" if won else "✗ Lost"
    print(f"\n{result}: Feedback - {json.dumps(feedback)}")
    
    time.sleep(0.5)



--- Auction 1 ---

BID REQUEST (OpenRTB):
{
  "id": "hP0yc3oZRRyoMszli49fnQ",
  "imp": [
    {
      "id": "1",
      "banner": {
        "w": 300,
        "h": 250,
        "format": [
          {
            "w": 300,
            "h": 250
          }
        ]
      },
      "tagid": "sidebar-ad-123",
      "bidfloor": 1.5,
      "bidfloorcur": "USD",
      "secure": 1,
      "ext": {
        "billing_id": [
          12345
        ]
      }
    }
  ],
  "site": {
    "id": "site-pub-123",
    "domain": "example-publisher.com",
    "page": "https://example-publisher.com/article",
    "name": "Publisher",
    "cat": null,
    "content": null
  },
  "device": {
    "ua": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X)",
    "ip": "192.168.1.100",
    "devicetype": 1,
    "make": "Apple",
    "model": "iPhone",
    "os": "iOS"
  },
  "user": {
    "id": "user-7e21e9a8",
    "buyeruid": "0af7ae4b-601a-4f96-ada5-3a335eb596b9",
    "keywords": ""
  },
  "at": 2,
  "tmax": 100,
  "

In [8]:

# Display metrics
print("\n" + "=" * 60)
print("PERFORMANCE METRICS")
print("=" * 60)
metrics = bidder.get_performance_metrics()
for key, value in metrics.items():
    print(f"{key.replace('_', ' ').title()}: {value}")



PERFORMANCE METRICS
Bids Submitted: 53
Bids Won: 27
Win Rate: 50.943396226415096
Total Spend: 163.35
Average Cpm: 6.05
Impressions Count: 27
